# Video Processing for BT Lab
This notebook to test risky code fragments
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [18]:
#import libraries
import os
import cv2
import json
import re
import datetime
import time
import numpy as np

In [19]:
"""delete unwanted files"""
# desired_path = r"D:\Christian\DI_centre_structured\DI_THERMAL_FLIR\3\2 meters\H\WOB\frames"
# os.chdir(desired_path)
# copied_files = [bmp_files for bmp_files in os.listdir() if "Copy" in bmp_files]
# copied_files

'delete unwanted files'

In [20]:
# for copy_file in copied_files:
#     os.remove(copy_file)

In [21]:
def convert_mp4_to_frames(mp4_file:str, output_dir:str):
    video = cv2.VideoCapture(mp4_file)
    os.makedirs(output_dir, exist_ok=True)
    frame_count = 0
    not_end_of_vid = True

    while not_end_of_vid:
        ret, frame = video.read()
        
        if not ret:
            not_end_of_vid = False
            break

        frame_file = os.path.join(output_dir, f"frame{frame_count:05d}.jpg")
        cv2.imwrite(frame_file, frame)
        frame_count += 1

        # if frame_count > 20:
        #     break

    video.release()

    print(f"Conversion complete: {frame_count} frames saved in {output_dir}")

In [22]:
def count_frames(video_path: str) -> int:
    frame_count = 0
    still_has_frames = True
    video = cv2.VideoCapture(video_path)

    if not video.isOpened():
        print("Error opening video file")
        return 2
    
    while still_has_frames:
        ret, frame = video.read()
        if not ret:
            still_has_frames = False
        else:
            frame_count += 1
    
    video.release()
    return frame_count

In [23]:
def get_video_properties(mp4_file: str, is_real=True) -> list:
    try:
        video_properties = []
        video = cv2.VideoCapture(mp4_file)
        
        if is_real:
            number_of_frames = count_frames(mp4_file)
            old_fps = video.get(cv2.CAP_PROP_FPS)
        else:
            number_of_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
            old_fps = video.get(cv2.CAP_PROP_FPS)
        
        length = number_of_frames / old_fps
        video_time = datetime.timedelta(seconds=length)
        video.release()
    except Exception as e:
        print(f"{type(e)} - from {mp4_file}")
    else:
        video_properties.append(number_of_frames)
        video_properties.append(old_fps)
        video_properties.append(length)
        video_properties.append(video_time)
    
    return video_properties

In [93]:
def down_sample_frames(frames, new_fps):
    if len(frames) <= new_fps:
        #need to "up sample"
        return frames

    down_sampled_list = []
    step = len(frames) / new_fps

    for fps in range(new_fps):
        start = int(fps * step)
        end = int((fps + 1) * step)
        
        if fps == new_fps - 1:
            # ensures that the last element includes any remaining elements
            end = len(frames)
        
        avg = sum(frames[start:end]) // (end - start)
        print(end, start, sum(frames[start:end]), avg)
        down_sampled_list.append(avg)

    return down_sampled_list

In [94]:
old_fps = 29
new_fps_lower = 10
new_fps_upper = 20
my_frames = np.arange(0, old_fps, dtype=int)

downsampled_list = down_sample_frames(my_frames, new_fps_upper)

1 0 0 0
2 1 1 1
4 2 5 2
5 4 4 4
7 5 11 5
8 7 7 7
10 8 17 8
11 10 10 10
13 11 23 11
14 13 13 13
15 14 14 14
17 15 31 15
18 17 17 17
20 18 37 18
21 20 20 20
23 21 43 21
24 23 23 23
26 24 49 24
27 26 26 26
29 27 55 27


In [95]:
missing_frames = []
for sample in my_frames:
    if sample not in downsampled_list:
        missing_frames.append(sample)

print(missing_frames, len(missing_frames))
print(downsampled_list)

[3, 6, 9, 12, 16, 19, 22, 25, 28] 9
[0, 1, 2, 4, 5, 7, 8, 10, 11, 13, 14, 15, 17, 18, 20, 21, 23, 24, 26, 27]


In [108]:
def up_sample_list(frames, new_fps):
    if new_fps <= len(frames):
        raise ValueError("Target length must be greater than the current list length.")

    frames_arr = frames
    frames_interp = np.linspace(0, len(frames) - 1, new_fps)
    up_sampled_list = np.interp(frames_interp, frames_arr, frames)
    rounded_up_sampled_list = up_sampled_list.astype(int)

    return rounded_up_sampled_list.tolist()


In [109]:
# Example usage:
old_fps = 18
new_fps_lower = 10
new_fps_upper = 20
my_frames = np.arange(0, old_fps, dtype=int)
upsampled_result = upsample_list(my_frames, new_fps_upper)
print(upsampled_result)

[0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
